# 1.Import Data

In [1]:
import numpy as np
import pickle
import sklearn
from tqdm import tqdm

In [2]:
y_name={
    "y_train":"train_y",
    "y_test":"test_y"
}

In [3]:
def open_pkl(file_name):
    all_data=dict()
    for each_key in file_name.keys():
        name = file_name[each_key]
        pickle_file = open('DataSet\\'+name+".pkl",mode="rb")
        data = pickle.load(pickle_file)
        pickle_file.close()
        all_data[each_key]=data
    return all_data

In [4]:
y_data = open_pkl(y_name)
y_train = y_data["y_train"]
y_test = y_data["y_test"]

In [5]:
y_train

array([0, 3, 0, ..., 0, 0, 0])

In [6]:
file_name0={
    "3turn":"vec_train_3_no_cor_glove_change",
    }
file_name1={
    "3turn":"vec_test_3_no_cor_glove_change",
}

In [7]:
#import data
def open_pkl_data(file_name):
    all_data=dict()
    for each_key in file_name.keys():
        name = file_name[each_key]
        pickle_file = open('DataSet\\'+name+".pkl",mode="rb")
        data = pickle.load(pickle_file)
        pickle_file.close()
        all_data[each_key]=data
    return all_data

In [8]:
train_data_overall = open_pkl_data(file_name0)
test_data_overall = open_pkl_data(file_name1)

In [9]:
original_train = train_data_overall["3turn"]
original_test = test_data_overall["3turn"]

# 2. Calculation of the average vectors of each sentence

In [10]:
length = len(original_train["overall"][0][0])
print(length)

300


In [11]:
def cal_average(all_data):
    output_dict=dict()
    for key in all_data.keys():
        sentences = list()
        for sentence in all_data[key]:
            sentence = np.array(sentence)
            length = len(sentence)
            sum_sen = sentence.sum(axis=0)
            aver_sen = sum_sen/length
            sentences.append(aver_sen)
        output_dict[key]=sentences
    
    return output_dict

In [12]:
ave_train = cal_average(original_train)
ave_test = cal_average(original_test)

In [13]:
length = len(ave_train["overall"][0])
print(length)

300


In [14]:
len(ave_train["overall"])

30160

# 3.SVM Model
## 3.1. build and fit model

In [15]:
from sklearn.svm import SVC
def svm_model(train_data,y_train):
    classifiers = dict()
    for key in tqdm(train_data.keys()):
        classifier = SVC(kernel = 'linear', random_state=0)
        classifier.fit(train_data[key],y_train)
        classifiers[key]=classifier
    
    return classifiers

In [16]:
classifiers = svm_model(ave_train,y_train)

100%|███████████████████████████████████████████████████████████████████████████████████| 3/3 [14:20<00:00, 286.85s/it]


In [17]:
# save model
def save_data(name,data):
    pickle_file = open('DataSet\\'+name+".pkl",mode='wb')
    pickle.dump(data,pickle_file)
    pickle_file.close()

In [18]:
# save model
save_data("SVM_comparison_glove",classifiers)

## 3.2. prediction

In [19]:
# function for getting prediction
def get_prediction(classifiers, ave_test):
    y_pred_dict= dict()
    list1 = list(classifiers.keys())
    list2 = list(ave_test.keys())
    length = len(list1)
    
    for i in tqdm(range(length)):
        y_pred_dict[list2[i]]=classifiers[list1[i]].predict(ave_test[list2[i]])
        
    return y_pred_dict

In [20]:
y_preds = get_prediction(classifiers,ave_test)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:22<00:00, 27.60s/it]


In [21]:
# function for getting the score of each prediction
def get_scores(classifiers, ave_test):
    y_score_dict= dict()
    list1 = list(classifiers.keys())
    list2 = list(ave_test.keys())
    length = len(list1)
    
    for i in tqdm(range(length)):
        y_score_dict[list2[i]]=classifiers[list1[i]].decision_function(ave_test[list2[i]])
        
    return y_score_dict

In [22]:
y_scores = get_scores(classifiers, ave_test)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [01:21<00:00, 27.31s/it]


In [23]:
# function for saving the predictions and scores
def save_prediction(name,data):
    pickle_file = open('Prediction\\'+name+".pkl",mode='wb')
    pickle.dump(data,pickle_file)
    pickle_file.close()

In [24]:
save_prediction("SVM_process3turn_glove_preds",y_preds)
save_prediction("SVM_process3turn_glove_scores",y_scores)

## 3.3. Evaluation: confusion matrix, accuracy, precision, recall, F1 measure

In [25]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
# print evaluation result
def get_evaluation(y_test,y_pred):
    ev = dict()
    ev["confusion_matrix"]=confusion_matrix(y_test,y_pred)
    ev["accuracy"] = accuracy_score(y_test,y_pred)
    ev["precision_macro"]=precision_score(y_test,y_pred,average="macro")
    ev["precision_micro"]=precision_score(y_test,y_pred,average="micro")
    ev["precision_weighted"]=precision_score(y_test,y_pred,average="weighted")
    ev["recall_macro"]=recall_score(y_test,y_pred,average="macro")
    ev["recall_micro"]=recall_score(y_test,y_pred,average="micro")
    ev["recall_weighted"]=recall_score(y_test,y_pred,average="weighted")
    ev["F1_score_macro"]=f1_score(y_test,y_pred,average="macro")
    ev["F1_score_micro"]=f1_score(y_test,y_pred,average="micro")
    ev["F1_score_weighted"]=f1_score(y_test,y_pred,average="weighted")
    
    for key in ev.keys():
        if key !="confusion_matrix":
            print("{a} is: {b}".format(a=key, b=ev[key]))
        else:
            print(ev[key])
    
    return ev

    

In [26]:
#"macro": "Calculate metrics for each label, and find their unweighted mean. ",
#"micro": "Calculate metrics globally by counting the total true positives, false negatives and false positives."
#"weighted":"Calculate metrics for each label, and find their average weighted by support (the number of true instances for each label). "

def get_evaluation_all(y_test, y_pred_dict):
    for key in tqdm(y_pred_dict.keys()):
        print("evaluation on data {0}".format(key))
        get_evaluation(y_test, y_pred_dict[key])
        print("***************************************")
    

In [27]:
get_evaluation_all(y_test,y_preds)

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 41.16it/s]

evaluation on data overall
[[4035  188  226  228]
 [ 145  116   11   12]
 [  92    4  139   15]
 [  75    7   15  201]]
accuracy is: 0.8152114721365039
precision_macro is: 0.523192131543693
precision_micro is: 0.8152114721365039
precision_weighted is: 0.8470010474209272
recall_macro is: 0.6254199673418377
recall_micro is: 0.8152114721365039
recall_weighted is: 0.8152114721365039
F1_score_macro is: 0.5621119871098925
F1_score_micro is: 0.8152114721365039
F1_score_weighted is: 0.8277106826352397
***************************************
evaluation on data double13
[[4041  197  205  234]
 [ 135  131   11    7]
 [  80    4  151   15]
 [  62    6   13  217]]
accuracy is: 0.8241060083499727
precision_macro is: 0.5448915251070608
precision_micro is: 0.8241060083499727
precision_weighted is: 0.8573421354104298
recall_macro is: 0.6643677298951334
recall_micro is: 0.8241060083499727
recall_weighted is: 0.8241060083499727
F1_score_macro is: 0.5904978570723562
F1_score_micro is: 0.8241060083499726
F

# 4. if just want to directly load the model

In [28]:
model_names = {
    "model_all":"SVM_comparison_glove"
}

In [29]:
model_data = open_pkl(model_names)
classifiers = model_data["model_all"]